## Feature Engineering and Selection: Distance

This notebook details another attmpt at feature engineering and selection for distance-related variables. It is motivated by the new descriptive analysis results on distance and the idea to use zone centroid coordinates as a features in the Random Forest model.

First, let's load the data, taking care to remove `Other` students.

In [2]:
import pandas as pd

df = pd.read_csv('../Data/SMTO_2015/SMTO_2015_Complete_Input.csv')
df = df[df['Level'] != 'Other']
df.head()

,Campus,Level,Status,Mode_Actual,Gender,Licence,Work,Age,HomeZone,Family,...,Domestic.OC,Admission_Avg.SG,Admission_Avg.SC,Admission_Avg.MI,Admission_Avg.YK,Admission_Avg.YG,Admission_Avg.RY,Admission_Avg.OC,Exp_Segment,Exp_Level
0,Scarborough (UTSC),UG,FT,Transit Bus,Female,0,Unknown,20,261,1,...,0.8998,0.893,0.841,0.83,0.817,0.817,0.84,0.824,0.944738,0.944738
1,Downtown Toronto (St. George),Grad,FT,Walk,Female,1,Unknown,25,71,0,...,0.6786,0.893,0.841,0.83,0.817,0.817,0.84,0.824,0.986085,0.986085
2,Downtown Toronto (St. George),UG,FT,Transit Bus,Female,1,Unknown,23,3714,1,...,0.8998,0.893,0.841,0.83,0.817,0.817,0.84,0.824,0.919270,0.919270
3,Downtown Toronto (St. George),UG,FT,Walk,Male,1,Unknown,20,74,0,...,0.8998,0.893,0.841,0.83,0.817,0.817,0.84,0.824,0.919270,0.919270
4,Downtown Toronto (St. George),Grad,FT,Walk,Male,1,Unknown,27,71,0,...,0.6786,0.893,0.841,0.83,0.817,0.817,0.84,0.824,0.986085,0.986085


Let us quickly look at the correlation of the distance columns.

In [4]:
df.iloc[:, 17:24].corr()

,Dist.SG,Dist.SC,Dist.MI,Dist.YK,Dist.YG,Dist.RY,Dist.OC
Dist.SG,1.000000,0.584396,0.588353,0.731334,0.864714,0.997410,0.998566
Dist.SC,0.584396,1.000000,-0.139397,0.511810,0.847196,0.618299,0.567580
Dist.MI,0.588353,-0.139397,1.000000,0.464808,0.272800,0.549017,0.600958
Dist.YK,0.731334,0.511810,0.464808,1.000000,0.817802,0.715375,0.712210
Dist.YG,0.864714,0.847196,0.272800,0.817802,1.000000,0.878822,0.848379
Dist.RY,0.997410,0.618299,0.549017,0.715375,0.878822,1.000000,0.996756
Dist.OC,0.998566,0.567580,0.600958,0.712210,0.848379,0.996756,1.000000


We notice that `Dist.SG`, `Dist.RY`, and `Dist.OC` are very highly correlated. This is not surprising as these campuses are in close proximity. Hence, we can try running models with only one of these columns included. Furthermore, the `Dist.YG` column is correlated with those three columns and with `Dist.SC`. We also try excluding that column from the model input.

In addition to changing the number of distance columns passed, we also engineer some additional features. We add six "flag" columns which indicate whether a student's HomeZone is within a certain distance of particular campuses. These thresholds were determined by observing trends found in the descriptive analysis. Finally, we also add flags for whether a student lives in the same zone as each campus.

In [5]:
# Adding flag columns for distances
df['SC<25'] = df['Dist.SC'] < 25
df['SG<10'] = df['Dist.SG'] < 10
df['SG<20'] = df['Dist.SG'] < 20
df['MI<10'] = df['Dist.MI'] < 10
df['MI<20'] = df['Dist.MI'] < 20
df['YK<20'] = df['Dist.YK'] < 20

# Adding flags for same zone as campus
school_codes = list(df['School_Codes'].unique())
for school in school_codes:
    if school == 'YG':
        continue
    df[school + '0'] = df['Dist.' + school] == 0
print(list(df.columns))

['Campus', 'Level', 'Status', 'Mode_Actual', 'Gender', 'Licence', 'Work', 'Age', 'HomeZone', 'Family', 'Cars', 'Children', 'Adults', 'Income', 'Mode', 'School_Codes', 'Segment', 'Dist.SG', 'Dist.SC', 'Dist.MI', 'Dist.YK', 'Dist.YG', 'Dist.RY', 'Dist.OC', 'WTT.SG', 'WTT.SC', 'WTT.MI', 'WTT.YK', 'WTT.YG', 'WTT.RY', 'WTT.OC', 'AIVTT.SG', 'AIVTT.SC', 'AIVTT.MI', 'AIVTT.YK', 'AIVTT.YG', 'AIVTT.RY', 'AIVTT.OC', 'TPTT.SG', 'TPTT.SC', 'TPTT.MI', 'TPTT.YK', 'TPTT.YG', 'TPTT.RY', 'TPTT.OC', 'Total.SG', 'Total.SC', 'Total.MI', 'Total.YK', 'Total.YG', 'Total.RY', 'Total.OC', 'UG.SG', 'UG.SC', 'UG.MI', 'UG.YK', 'UG.YG', 'UG.RY', 'UG.OC', 'Grad.SG', 'Grad.SC', 'Grad.MI', 'Grad.YK', 'Grad.YG', 'Grad.RY', 'Grad.OC', 'Tuition.SG', 'Tuition.SC', 'Tuition.MI', 'Tuition.YK', 'Tuition.YG', 'Tuition.RY', 'Tuition.OC', 'Domestic.SG', 'Domestic.SC', 'Domestic.MI', 'Domestic.YK', 'Domestic.YG', 'Domestic.RY', 'Domestic.OC', 'Admission_Avg.SG', 'Admission_Avg.SC', 'Admission_Avg.MI', 'Admission_Avg.YK', 'Admiss

Now, let us load coordinate and planning district information. To avoid scaling issues, we normalize the coordinates so that the values are between 0 and 1, inclusive. We also plot their correlations.

In [6]:
# Load zone coordinates
zones = pd.read_csv('../Data/Zones.csv')
zones.set_index('Zone#', inplace=True)

# Normalize from 0 to 1
zones['X'] = (zones['X'] - zones['X'].min()) / (zones['X'].max() - zones['X'].min())
zones['Y'] = (zones['Y'] - zones['Y'].min()) / (zones['Y'].max() - zones['Y'].min())
zones.corr()

,PD,X,Y
PD,1.000000,-0.456036,-0.371999
X,-0.456036,1.000000,0.633247
Y,-0.371999,0.633247,1.000000


Interestingly, the X- and Y-coordinates are moderately positively correlated. This might be indicative of the shape of the GTA being tilted from the southwest to the northeast due to Lake Ontario, as well as York Region's large extent into the northeast.

Let us add this zone information to our dataframe.

In [7]:
# Add zone information to df
temp = pd.DataFrame([[a[i] for a in (zones['PD'], zones['X'], zones['Y'])] for i in df['HomeZone']], columns=['PD', 'X', 'Y'], index=df.index)
df = pd.concat((df, temp), axis=1)

In [111]:
df.head()

,Campus,Level,Status,Mode_Actual,Gender,Licence,Work,Age,HomeZone,Family,...,Admission_Avg.MI,Admission_Avg.YK,Admission_Avg.YG,Admission_Avg.RY,Admission_Avg.OC,Exp_Segment,Exp_Level,PD,X,Y
0,Scarborough (UTSC),UG,FT,Transit Bus,Female,0,Unknown,20,261,1,...,0.83,0.817,0.817,0.84,0.824,0.944738,0.944738,6,0.452150,0.469128
1,Downtown Toronto (St. George),Grad,FT,Walk,Female,1,Unknown,25,71,0,...,0.83,0.817,0.817,0.84,0.824,0.986085,0.986085,1,0.428078,0.447426
2,Downtown Toronto (St. George),UG,FT,Transit Bus,Female,1,Unknown,23,3714,1,...,0.83,0.817,0.817,0.84,0.824,0.919270,0.919270,36,0.322351,0.385293
3,Downtown Toronto (St. George),UG,FT,Walk,Male,1,Unknown,20,74,0,...,0.83,0.817,0.817,0.84,0.824,0.919270,0.919270,1,0.426631,0.440360
4,Downtown Toronto (St. George),Grad,FT,Walk,Male,1,Unknown,27,71,0,...,0.83,0.817,0.817,0.84,0.824,0.986085,0.986085,1,0.428078,0.447426


In [8]:
df = pd.get_dummies(df, columns = ['Level', 'Status','Work'])

In [9]:
df

,Campus,Mode_Actual,Gender,Licence,Age,HomeZone,Family,Cars,Children,Adults,...,X,Y,Level_Grad,Level_UG,Status_FT,Status_PT,Work_FT,Work_NW,Work_PT,Work_Unknown
0,Scarborough (UTSC),Transit Bus,Female,0,20,261,1,1,3,4,...,0.452150,0.469128,0,1,1,0,0,0,0,1
1,Downtown Toronto (St. George),Walk,Female,1,25,71,0,0,0,2,...,0.428078,0.447426,1,0,1,0,0,0,0,1
2,Downtown Toronto (St. George),Transit Bus,Female,1,23,3714,1,1,0,4,...,0.322351,0.385293,0,1,1,0,0,0,0,1
3,Downtown Toronto (St. George),Walk,Male,1,20,74,0,0,0,4,...,0.426631,0.440360,0,1,1,0,0,0,0,1
4,Downtown Toronto (St. George),Walk,Male,1,27,71,0,0,0,2,...,0.428078,0.447426,1,0,1,0,0,0,0,1
5,Downtown Toronto (St. George),Walk,Female,0,20,72,0,0,0,3,...,0.425183,0.446553,0,1,1,0,0,0,1,0
6,Scarborough (UTSC),Transit Bus,Female,0,18,600,1,2,1,4,...,0.463268,0.537959,0,1,1,0,0,0,0,1
7,Scarborough (UTSC),Car - Driver alone,Female,1,21,3420,1,2,0,5,...,0.321390,0.498651,0,1,1,0,0,0,0,1
8,Downtown Toronto (St. George),Bicycle,Female,1,33,113,0,1,0,2,...,0.412570,0.429943,1,0,1,0,0,0,0,1
9,Downtown Toronto (St. George),GO Train,Female,1,31,1031,0,1,0,2,...,0.522240,0.555820,1,0,1,0,0,0,0,1


With the data prepared, we can begin preparing the Random Forest classifier. First, we import the relevant packages.

In [10]:
# Machine learning packages
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
#from sklearn.inspection import permutation_importance
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import matthews_corrcoef

def average(l):
    return sum(l) / len(l)

We declare variables for the distance-related columns so we can pass them into the model.

In [11]:
std_dists = df.iloc[:, 17:24]
dist_flags = df.iloc[:, 89:95]
same_zone_flags = df.iloc[:, 96:101]
four_dists = df.iloc[:, 17:21] # No OC, RY, YG
five_dists = df.iloc[:, 17:22] # No OC, RY
coords = df[['X', 'Y']]
three_dists = df.iloc[:, 18:21]
three_dists_b = df.iloc[:, 17:20]
coords_fam = df[['X', 'Y', 'Family']]
dist_fam = df[['Dist.MI','Dist.SC','Dist.YK', 'Family']]
coords_level = df[['X', 'Y', 'Level_Grad', 'Level_UG']] 
dist_level = df[['Dist.MI','Dist.SC','Dist.YK',  'Level_Grad', 'Level_UG']]
coords_status = df[['X', 'Y', 'Status_FT', 'Status_PT']]
dist_status = df[['Dist.MI','Dist.SC','Dist.YK', 'Status_FT', 'Status_PT']]
dist_level_family = df[['Dist.MI','Dist.SC','Dist.YK',  'Level_Grad', 'Level_UG', 'Family']]
coords_level_family = df[['X', 'Y', 'Level_Grad', 'Level_UG', 'Family']] 
dist_status_family = df[['Dist.MI','Dist.SC','Dist.YK', 'Family', 'Status_FT', 'Status_PT']]
coords_status_family = df[['X', 'Y', 'Status_FT', 'Status_PT', 'Family']] 
dist_status_level = df[['Dist.MI','Dist.SC','Dist.YK',  'Level_Grad', 'Level_UG','Status_FT', 'Status_PT']]
coords_status_level = df[['X', 'Y', 'Status_FT', 'Status_PT', 'Level_Grad', 'Level_UG']] 
dist_status_level_family = df[['Dist.MI','Dist.SC','Dist.YK',  'Level_Grad', 'Level_UG','Status_FT', 'Status_PT', 'Family']]
coords_status_level_family = df[['X', 'Y', 'Status_FT', 'Status_PT', 'Level_Grad', 'Level_UG','Family']] 

cars = df['Cars']
licence = df['Licence']

We also prepare a dataframe to store the model results, including the following metrics:
- Accuracy: The accuracy of the model as calculated by `sklearn`, also the micro precision/recall/F-1 score  
- Prec, Rec, F1: The macro precision, recall, and F-1 score  
- MCC: The Matthews Correlation Coefficient
- APO (Average Probabilities of Observations): The average predicted probabilities produced by the classifier of observed campus choices for the testing data.

In [16]:
# Prepare results dataframe
metric_names = ['Acc', 'Prec', 'Rec', 'F1', 'MCC', 'APO']
results = pd.DataFrame(columns=['Model'] + metric_names)

Finally, let us run our models. For each model, we average the metrics across 10 trials.

In [17]:
# Prepare classifier
rf = RandomForestClassifier(n_estimators=100)
y = df['School_Codes']
num_trials = 20

# Run models
for x_temp, name in (#(std_dists, 'All_Dists'),
#(pd.concat((std_dists, df['Work_FT'],df['Work_PT'],df['Work_NW'],df['Work_Unknown']), axis=1), "All_Dists + Work"),
#(pd.concat((three_dists, df['Work_FT'],df['Work_PT'],df['Work_NW'],df['Work_Unknown']), axis=1), "Three_Dists + Work"),
#(pd.concat((coords, df['Work_FT'],df['Work_PT'],df['Work_NW'],df['Work_Unknown']), axis=1), "Coords + Work"),
#(pd.concat((std_dists, df['Family'], df['Work_FT'],df['Work_PT'],df['Work_NW'],df['Work_Unknown']), axis=1), "All_Dists + Family + Work"),
#(pd.concat((three_dists, df['Family'], df['Work_FT'],df['Work_PT'],df['Work_NW'],df['Work_Unknown']), axis=1), "Three_Dists + Family + Work"),
#(pd.concat((coords,  df['Family'], df['Work_FT'],df['Work_PT'],df['Work_NW'],df['Work_Unknown']), axis=1), "Coords + Family + Work"),  
#(pd.concat((std_dists, df['Level_UG'], df['Level_Grad'], df['Work_FT'],df['Work_PT'],df['Work_NW'],df['Work_Unknown']), axis=1), "All_Dists + Level + Work"),
#(pd.concat((three_dists, df['Level_UG'], df['Level_Grad'], df['Work_FT'],df['Work_PT'],df['Work_NW'],df['Work_Unknown']), axis=1), "Three_Dists + Level + Work"),
#(pd.concat((coords,  df['Level_UG'], df['Level_Grad'], df['Work_FT'],df['Work_PT'],df['Work_NW'],df['Work_Unknown']), axis=1), "Coords + Level + Work"),
#(coords_fam, 'Coords + Family'),
#(coords_level, 'Coords + Level'),
#(coords_status, 'Coords + Status'),
#(dist_fam, 'Three_Dists + Family'),
#(dist_level, 'Three_Dists + Level'),
#(dist_status, 'Three_Dists + Status'),
#(three_dists, 'Three_Dists'),
#(dist_level_family,'Three_Dists + Family + Level'),
#(coords_level_family,'Coords + Family + Level'),
#(dist_status_family,'Three_Dists + Family + Status'),
#(coords_status_family,'Coords + Family + Status'),
#(dist_status_level,'Three_Dists + Level + Status'),
#(coords_status_level,'Coords + Level + Status'),
#(dist_status_level_family ,'Three_Dists + Family + Level + Status'),
#(coords_status_level_family ,'Coords + Family + Level + Status'),
#(coords, 'Coords'),
#(pd.concat((std_dists, df['Family']), axis=1), "All_Dists + Family"),
#(pd.concat((std_dists, df['Level_Grad'], df['Level_UG']), axis=1), "All_Dists + Level"),
#(pd.concat((std_dists, df['Status_FT'], df['Status_PT']), axis=1), "All_Dists + Status"),
#(pd.concat((std_dists, df['Family'], df['Level_UG'], df['Level_Grad']), axis=1), "All_Dists + Family + Level"),
#(pd.concat((std_dists, df['Family'], df['Status_FT'], df['Status_PT']), axis=1), "All_Dists + Family + Status"), 
#(pd.concat((std_dists, df['Status_FT'], df['Status_PT'], df['Level_UG'], df['Level_Grad']), axis=1), "All_Dists + Status + Level"),
#(pd.concat((std_dists, df['Status_FT'], df['Status_PT'], df['Level_UG'], df['Level_Grad'], df['Family']), axis=1), "All_Dists + Family + Status + Level"),
(pd.concat((std_dists, df['Cars']), axis=1), "All_Dists + Cars"),
(pd.concat((std_dists, df['Licence']), axis=1), "All_Dists + Licence"),
(pd.concat((three_dists, df['Cars']), axis=1), "Three_Dists + Cars"),
(pd.concat((three_dists, df['Licence']), axis=1), "Three_Dists + Licence"),
(pd.concat((coords, df['Cars']), axis=1), "Coords + Cars"),
(pd.concat((coords, df['Licence']), axis=1), "Coords + Licence"),
(pd.concat((std_dists, df['Cars'], df['Licence']), axis=1), "All_Dists + Cars + Licence"),
(pd.concat((three_dists, df['Cars'], df['Licence']), axis=1), "Three_Dists + Cars + Licence"),
(pd.concat((coords, df['Cars'], df['Licence']), axis=1), "Coords + Cars + Licence"),
#(three_dists_b, 'Three_b'),
#(five_dists, 'Five'),
#(four_dists, 'Four'),
#(dist_flags, 'Flags'),
#(same_zone_flags, 'SameZone'),
#(df['HomeZone'].values.reshape(-1, 1), 'HZ'),
#(df['PD'].values.reshape(-1, 1), 'PD'),
#(df['HomeZone'].isnull().values.reshape(-1,1), "Null"),
#(pd.concat((four_dists, df['PD']), axis=1), "Four+PD"),
#(pd.concat((coords, std_dists), axis=1), "Std+Coords"),
#(pd.concat((coords, five_dists), axis=1), "Five+Coords"),
#(pd.concat((coords, four_dists), axis=1), "Four+Coords"),
#(pd.concat((coords, std_dists, df['HomeZone']), axis=1), "Std+Coords+HZ"),
#(pd.concat((coords, five_dists, df['HomeZone']), axis=1), "Five+Coords+HZ"),
#(pd.concat((coords, four_dists, df['HomeZone']), axis=1), "Four+Coords+HZ"),
#(pd.concat((coords, four_dists), axis=1), "Four+Coords+HZ+PD"),
#(pd.concat((coords, four_dists, df['HomeZone']), axis=1), "Four+Coords+PD"),
#(pd.concat((df['PD'], dist_flags), axis=1), "PD+Flags"),
#(pd.concat((four_dists, dist_flags), axis=1), "Four+Flags"),
#(pd.concat((dist_flags, std_dists), axis=1), "Std+Flags"),
#(pd.concat((dist_flags, five_dists), axis=1), "Five+Flags"),
(pd.concat((std_dists, dist_flags, same_zone_flags, coords, df['HomeZone'], df['PD']), axis=1), "Full")):
    # Prepare metrics
    #metrics = {}
    #metrics_list = [None,None,None,None,None,None]
    #for metric in metric_names:
    #    metrics[metric] = []
    
    # Run trials
    for i in range(num_trials):
        metrics_list = [None,None,None,None,None,None]
        
        # Split data and run model
        X_train, X_test, y_train, y_test = train_test_split(x_temp, y, test_size=0.3)
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_test)
        
        # Metrics
        metrics_list[0] = (rf.score(X_test, y_test))
        metrics_list[1] = precision_recall_fscore_support(y_test, y_pred, average = 'macro')[0]
        metrics_list[2] = precision_recall_fscore_support(y_test, y_pred, average = 'macro')[1]
        metrics_list[3] = precision_recall_fscore_support(y_test, y_pred, average = 'macro')[2]
        metrics_list[4] = (matthews_corrcoef(y_test, y_pred))
        
        # APO
        schools = list(rf.classes_)
        probs = pd.concat((y_test.reset_index(drop=True), pd.DataFrame(rf.predict_proba(X_test))), axis=1)
        metrics_list[5] = (probs.apply(lambda z: z[schools.index(z.School_Codes)], axis=1).mean())
    
        # Add results to dataframe
        #ave_metrics = [average(v) for v in metrics.values()]
        #results.loc[len(results)] = [name, num_trials] + ave_metrics
        results.loc[len(results)] = [name] + metrics_list

The last model we ran was the "Full" model which included all the relevant columns. We can take a look at the feature importances from the last trial of this model:

In [25]:
#pd.DataFrame({'Features' : x_temp.columns, 'Importance': rf.feature_importances_}).sort_values('Importance', ascending=False)

It seems that the `HomeZone` column, coordinates, and distances were most important in this model. The planning district and flags were less important.

We can look at a correlation matrix for the different metrics.

In [26]:
#results.corr()

Interestingly, the correlation between Accuracy and APO is only 0.657. Macro F-1 score was quite closely correlated with APO so we may prioritize it as a computationally faster and more well-known indicator of effectiveness in the future.

At this point, we should look at the results for our different models:

In [18]:
results

,Model,Acc,Prec,Rec,F1,MCC,APO
0,All_Dists + Cars,0.449629,0.311195,0.303958,0.304092,0.232975,0.391002
1,All_Dists + Cars,0.445218,0.310329,0.288067,0.293645,0.225525,0.386943
2,All_Dists + Cars,0.443825,0.313912,0.298438,0.301154,0.229377,0.389687
3,All_Dists + Cars,0.427577,0.309239,0.290562,0.295473,0.208637,0.380391
4,All_Dists + Cars,0.451718,0.317846,0.299447,0.302003,0.234414,0.394594
5,All_Dists + Cars,0.445450,0.308970,0.299078,0.299910,0.234540,0.388993
6,All_Dists + Cars,0.432916,0.296888,0.284242,0.288383,0.215848,0.381951
7,All_Dists + Cars,0.455664,0.322997,0.315446,0.316930,0.239971,0.397395
8,All_Dists + Cars,0.441040,0.305268,0.289946,0.291980,0.224503,0.391640
9,All_Dists + Cars,0.442665,0.323541,0.300307,0.305113,0.225952,0.388384


In [19]:
results.groupby('Model').mean().sort_values('APO', ascending=False)

,Acc,Prec,Rec,F1,MCC,APO
Model,,,,,,
All_Dists + Cars,0.444522,0.309323,0.295745,0.298239,0.228034,0.389631
Coords + Cars,0.447911,0.315709,0.297321,0.301112,0.231583,0.389244
All_Dists + Licence,0.449803,0.318901,0.296506,0.300906,0.233755,0.388602
Three_Dists + Cars,0.443152,0.307168,0.293233,0.295797,0.225138,0.388525
Full,0.461571,0.344572,0.301104,0.305199,0.242763,0.388191
Coords + Licence,0.449687,0.314467,0.295614,0.298842,0.231401,0.387970
Coords + Cars + Licence,0.437581,0.306987,0.291657,0.295318,0.219386,0.387832
All_Dists + Cars + Licence,0.434738,0.300783,0.292244,0.293948,0.217911,0.387825
Three_Dists + Cars + Licence,0.436757,0.301638,0.289514,0.292298,0.218583,0.386865


In [125]:
results.groupby('Model').mean().to_csv('Variable_Runs_Results.csv')

Notice that there are significant fluctuations in these metrics across trials. Because of this, we are considering all the models with 0.38 < APO < 0.39 to be similarly effective. These models include any combination of distance columns, home zone, and/or coordinates. The different combinations of these variables, or the inclusion of flags, planning district, etc., did not seem to cause significant increases in effectiveness.

Interestingly, the "PD + Flags" model offered a significant imporvement on both the "PD" and "Flags" models individually, approaching the accuracy of the stronger models. Of note, however, was that this model had a very high micro accuracy.

Finally, it seems that including only four or five distance columns results in models that are no less effective than for all seven distance models. This might be an approach we will keep in the future.